<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [47]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
##### Your Code Here #####
df = pd.read_csv('./WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df = df.drop('customerID', axis=1)

In [5]:
df = df[df['TotalCharges'] != ' ']

In [6]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [7]:
categorical = df.select_dtypes(include='object')

In [8]:
categorical = categorical.drop(columns=['Churn'])

In [9]:
categorical.columns

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [10]:
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

In [11]:
import category_encoders as ce

enc = ce.OrdinalEncoder()

df[categorical.columns] = enc.fit_transform(df[categorical.columns])

In [12]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85,0
1,2,0,2,1,34,2,2,1,2,2,2,1,1,1,2,2,2,56.95,1889.50,0
2,2,0,2,1,2,2,2,1,2,1,1,1,1,1,1,1,2,53.85,108.15,1
3,2,0,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,1840.75,0
4,1,0,2,1,2,2,2,2,1,2,1,1,1,1,1,1,1,70.70,151.65,1


In [13]:
x = df.drop('Churn', axis=1)
y = df['Churn']

In [14]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer().fit(x)

x = transformer.transform(x)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y)

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5274, 19), (1758, 19), (5274,), (1758,))

In [32]:
y_train.value_counts()

0    3905
1    1369
Name: Churn, dtype: int64

In [41]:
3905 / 5274

0.7404247250663633

In [42]:
y_test.value_counts()

0    1258
1     500
Name: Churn, dtype: int64

In [43]:
1258 / 1758

0.7155858930602957

In [52]:
input_dim = X_train.shape[1]
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=input_dim, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [32,34,36,38,40],
             'epochs': [100, 150, 200, 250]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print(grid_result)

GridSearchCV(cv=None, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd1b87d03c8>,
             iid='deprecated', n_jobs=-1,
             param_grid={'batch_size': [32, 34, 36, 38, 40],
                         'epochs': [100, 150, 200, 250]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)


In [53]:
grid_result.best_score_, grid_result.best_params_

(0.7912431120872497, {'batch_size': 38, 'epochs': 250})

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=38, epochs=250)

In [54]:
cross_val_score(model, X_train, y_train, cv=4)

array([0.74374527, 0.73692191, 0.72534144, 0.74734443])

In [55]:
cross_val_score(model, X_test, y_test, cv=4)

array([0.73409092, 0.69772726, 0.68564922, 0.72665149])

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?